# CollecTRI_01_resource_aggregation  
Test translated python code from original [CollecTRI_01_resource_aggregation.R](https://github.com/saezlab/CollecTRI/blob/main/scripts/CollecTRI/01_resource_aggregation.R) to retrieve:  
* Number of activating and repressing interactions per TF-target pair
* TF class: Lambert, coTF etc

In [1]:
import os
import pandas as pd
import re

## Load and prepare data

In [2]:
# Load and prepare data
collecTRI_raw = pd.read_csv("CollecTRI_source.tsv", sep="\t", dtype="str")

print(collecTRI_raw.shape)
collecTRI_raw.head()

(51850, 55)


,TF:TG,Transcription Factor (Associated Gene Name),Target Gene (Associated Gene Name),[ExTRI] Confidence,[ExTRI] PMID,[ExTRI] present,[HTRI] present,[HTRI] Technique,[HTRI] PMID,[HTRI] Confidence,...,[DoRothEA_A] PMID,[DoRothEA_A] Directed,[DoRothEA_A] Effect,Lambert,Lovering,GO:0003700,GO:0140223,GO:0003712,TFClass,Auto-regulation
0,MYC:TERT,MYC,TERT,High,10022128;10022128;10022128;10491298;10491298;1...,ExTRI,HTRI,Chromatin Immunoprecipitation|Chromatin Immuno...,11274400|12695333,High|High,...,11274400;12695333,TRUE,Stimulate,Lambert,Lovering,GO:0003700,NaN,NaN,TFClass,NaN
1,AP1:CAT,AP1,CAT,High,10022519;10329043;12036993;12538496;17935786;7...,ExTRI,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,Lambert,Lovering,GO:0003700,NaN,NaN,TFClass,NaN
2,SPI1:BGLAP,SPI1,BGLAP,Low,10022617,ExTRI,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,Lambert,Lovering,GO:0003700,NaN,NaN,TFClass,NaN
3,AP1:ADAM2,AP1,ADAM2,High,10022869;10644332;11741970;11741970;18045590;1...,ExTRI,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,Lambert,Lovering,GO:0003700,NaN,NaN,TFClass,NaN
4,AP1:JUN,AP1,JUN,High,10022869;10022869;10037172;10208431;10366004;1...,ExTRI,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,Lambert,Lovering,GO:0003700,NaN,NaN,TFClass,NaN


In [3]:
# Checkout columns
collecTRI_raw.columns.tolist()

['TF:TG',
 'Transcription Factor (Associated Gene Name)',
 'Target Gene (Associated Gene Name)',
 '[ExTRI] Confidence',
 '[ExTRI] PMID',
 '[ExTRI] present',
 '[HTRI] present',
 '[HTRI] Technique',
 '[HTRI] PMID',
 '[HTRI] Confidence',
 '[TRRUST] present',
 '[TRRUST] Regulation',
 '[TRRUST] PMID',
 '[TFactS] present',
 '[TFactS] Sign',
 '[TFactS] Species',
 '[TFactS] Source',
 '[TFactS] PMID',
 '[TFactS] Confidence',
 '[GOA] present',
 '[GOA] Sign',
 '[GOA] PMID',
 '[IntAct] present',
 '[IntAct] PMID',
 '[IntAct] Method ID',
 '[SIGNOR] present',
 '[SIGNOR] Effect',
 '[SIGNOR] Sign',
 '[SIGNOR] PMID',
 '[CytReg] present',
 '[CytReg] Assay type',
 '[CytReg] species',
 '[CytReg] Activation/Repression',
 '[CytReg] PMID',
 '[CytReg] Year of publication',
 '[GEREDB] present',
 '[GEREDB] Effect',
 '[GEREDB] PMID',
 '[NTNU Curated] present',
 '[NTNU Curated] Sign',
 '[NTNU Curated] PMID',
 '[Pavlidis2021] present',
 '[Pavlidis2021] PMID',
 '[Pavlidis2021] Mode of action',
 '[DoRothEA_A] present

In [107]:
# Unify column names across resources
collecTRI_raw.columns = (
    collecTRI_raw.columns.str.replace(r" ", "_", regex=True)
    .str.replace(r"[", "", regex=True).str.replace(r"]", "", regex=True)
    .str.replace(r"(", "", regex=True).str.replace(r")", "", regex=True)
    .str.replace(r"SIGNOR_Effect", "SIGNOR_effectType")
    .str.replace(r"Effect", "Regulation")
    .str.replace(r"Sign", "Regulation")
    .str.replace(r"Activation/Repression", "Regulation", regex=True)
    .str.replace(r"Mode_of_action", "Regulation", regex=True)
    .str.replace(r":", "_")
)


collecTRI_raw.columns.tolist()

['Transcription_Factor_Associated_Gene_Name',
 'Target_Gene_Associated_Gene_Name',
 'ExTRI_Confidence',
 'ExTRI_PMID',
 'ExTRI_present',
 'HTRI_present',
 'HTRI_Technique',
 'HTRI_PMID',
 'HTRI_Confidence',
 'TRRUST_present',
 'TRRUST_Regulation',
 'TRRUST_PMID',
 'TFactS_present',
 'TFactS_Regulation',
 'TFactS_Species',
 'TFactS_Source',
 'TFactS_PMID',
 'TFactS_Confidence',
 'GOA_present',
 'GOA_Regulation',
 'GOA_PMID',
 'IntAct_present',
 'IntAct_PMID',
 'IntAct_Method_ID',
 'SIGNOR_present',
 'SIGNOR_effectType',
 'SIGNOR_Regulation',
 'SIGNOR_PMID',
 'CytReg_present',
 'CytReg_Assay_type',
 'CytReg_species',
 'CytReg_Regulation',
 'CytReg_PMID',
 'CytReg_Year_of_publication',
 'GEREDB_present',
 'GEREDB_Regulation',
 'GEREDB_PMID',
 'NTNU_Curated_present',
 'NTNU_Curated_Regulation',
 'NTNU_Curated_PMID',
 'Pavlidis2021_present',
 'Pavlidis2021_PMID',
 'Pavlidis2021_Regulation',
 'DoRothEA_A_present',
 'DoRothEA_A_PMID',
 'DoRothEA_A_Directed',
 'DoRothEA_A_Regulation',
 'Lamber

In [5]:
# Quick description of dataset: check missing values
collecTRI_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51850 entries, 0 to 51849
Data columns (total 55 columns):
 #   Column                                     Non-Null Count  Dtype 
---  ------                                     --------------  ----- 
 0   TF_TG                                      51850 non-null  object
 1   Transcription_Factor_Associated_Gene_Name  51850 non-null  object
 2   Target_Gene_Associated_Gene_Name           51850 non-null  object
 3   ExTRI_Confidence                           33670 non-null  object
 4   ExTRI_PMID                                 33670 non-null  object
 5   ExTRI_present                              33670 non-null  object
 6   HTRI_present                               2346 non-null   object
 7   HTRI_Technique                             2346 non-null   object
 8   HTRI_PMID                                  2346 non-null   object
 9   HTRI_Confidence                            2346 non-null   object
 10  TRRUST_present                    

In [6]:
# Set 'TF_TG' as the index
collecTRI_raw.set_index("TF_TG", inplace=True)

In [7]:
# Check resources with PMID information
resources = [
    resource.replace("_PMID", "")
    for resource in collecTRI_raw.columns
    if re.search(r"PMID", resource)
]

print(len(resources))
resources

12


['ExTRI',
 'HTRI',
 'TRRUST',
 'TFactS',
 'GOA',
 'IntAct',
 'SIGNOR',
 'CytReg',
 'GEREDB',
 'NTNU_Curated',
 'Pavlidis2021',
 'DoRothEA_A']

In [8]:
# Check resources with Regulation information (Not all resources have regulation ifonfo)
resources_reg = [
    resource.replace("_Regulation", "")
    for resource in collecTRI_raw.columns
    if re.search(r"Regulation", resource)
]

print(len(resources_reg))
resources_reg

9


['TRRUST',
 'TFactS',
 'GOA',
 'SIGNOR',
 'CytReg',
 'GEREDB',
 'NTNU_Curated',
 'Pavlidis2021',
 'DoRothEA_A']

In [9]:
# Filter for dataframe containing only PMIDs and associated modes of regulation
df_merge = pd.merge(collecTRI_raw.filter(regex="PMID"), collecTRI_raw.filter(regex="Regulation"), left_index=True, right_index=True)

df_merge.reset_index(inplace=True)
df_merge

,TF_TG,ExTRI_PMID,HTRI_PMID,TRRUST_PMID,TFactS_PMID,GOA_PMID,IntAct_PMID,SIGNOR_PMID,CytReg_PMID,GEREDB_PMID,...,DoRothEA_A_PMID,TRRUST_Regulation,TFactS_Regulation,GOA_Regulation,SIGNOR_Regulation,CytReg_Regulation,GEREDB_Regulation,NTNU_Curated_Regulation,Pavlidis2021_Regulation,DoRothEA_A_Regulation
0,MYC:TERT,10022128;10022128;10022128;10491298;10491298;1...,11274400|12695333,10022128;15595642;18226852;18754863;21132266;2...,14519204,NaN,NaN,NaN,NaN,NaN,...,11274400;12695333,Activation|Unknown,UP,UP,NaN,NaN,NaN,+|-|||,not_applicable|activation|activation|activation,Stimulate
1,AP1:CAT,10022519;10329043;12036993;12538496;17935786;7...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,SPI1:BGLAP,10022617,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,AP1:ADAM2,10022869;10644332;11741970;11741970;18045590;1...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,AP1:JUN,10022869;10022869;10037172;10208431;10366004;1...,NaN,9502786|19671687,|||,NaN,NaN,NaN,NaN,NaN,...,NaN,Activation|Unknown,|||,NaN,NaN,NaN,NaN,-|+||||+,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51845,TWIST1:MDM2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Unknown
51846,USF1:NUP188,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,22951020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Stimulate
51847,VDR:ELL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Unknown
51848,VDR:PTK2B,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Unknown


In [10]:
# Homogenize regulation modes
# NOTE: This does not work everywhere so perhaps do it later
df_merge.replace(["+", "activation", "Activation", "positive", "Stimulate", "UP"], 'activation', inplace=True)
df_merge.replace(["-", "DOWN", "Inhibit", "negative","Repression"], "repression", inplace=True)
df_merge.replace(["", "?", "+_-", "not_applicable","Unknown"], "unknown", inplace=True)

df_merge

,TF_TG,ExTRI_PMID,HTRI_PMID,TRRUST_PMID,TFactS_PMID,GOA_PMID,IntAct_PMID,SIGNOR_PMID,CytReg_PMID,GEREDB_PMID,...,DoRothEA_A_PMID,TRRUST_Regulation,TFactS_Regulation,GOA_Regulation,SIGNOR_Regulation,CytReg_Regulation,GEREDB_Regulation,NTNU_Curated_Regulation,Pavlidis2021_Regulation,DoRothEA_A_Regulation
0,MYC:TERT,10022128;10022128;10022128;10491298;10491298;1...,11274400|12695333,10022128;15595642;18226852;18754863;21132266;2...,14519204,NaN,NaN,NaN,NaN,NaN,...,11274400;12695333,Activation|Unknown,activation,activation,NaN,NaN,NaN,+|-|||,not_applicable|activation|activation|activation,activation
1,AP1:CAT,10022519;10329043;12036993;12538496;17935786;7...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,SPI1:BGLAP,10022617,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,AP1:ADAM2,10022869;10644332;11741970;11741970;18045590;1...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,AP1:JUN,10022869;10022869;10037172;10208431;10366004;1...,NaN,9502786|19671687,|||,NaN,NaN,NaN,NaN,NaN,...,NaN,Activation|Unknown,|||,NaN,NaN,NaN,NaN,-|+||||+,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51845,TWIST1:MDM2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,unknown
51846,USF1:NUP188,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,22951020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,activation
51847,VDR:ELL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,unknown
51848,VDR:PTK2B,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,unknown


In [11]:
# TEST MELTING OF DATAFRAME
# DO Subset of df_merge for test
df_merge = df_merge[:3]
display(df_merge)


,TF_TG,ExTRI_PMID,HTRI_PMID,TRRUST_PMID,TFactS_PMID,GOA_PMID,IntAct_PMID,SIGNOR_PMID,CytReg_PMID,GEREDB_PMID,...,DoRothEA_A_PMID,TRRUST_Regulation,TFactS_Regulation,GOA_Regulation,SIGNOR_Regulation,CytReg_Regulation,GEREDB_Regulation,NTNU_Curated_Regulation,Pavlidis2021_Regulation,DoRothEA_A_Regulation
0,MYC:TERT,10022128;10022128;10022128;10491298;10491298;1...,11274400|12695333,10022128;15595642;18226852;18754863;21132266;2...,14519204,NaN,NaN,NaN,NaN,NaN,...,11274400;12695333,Activation|Unknown,activation,activation,NaN,NaN,NaN,+|-|||,not_applicable|activation|activation|activation,activation
1,AP1:CAT,10022519;10329043;12036993;12538496;17935786;7...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,SPI1:BGLAP,10022617,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
# Define columns containing PMID and columns containing Regulation
## NOTE: this code can be used already before when merging dataframes for PMID and for Regulation

pmid_cols_ls = df_merge.columns[df_merge.columns.str.contains("PMID")].tolist()
print(pmid_cols_ls)

reg_cols_ls = df_merge.columns[df_merge.columns.str.contains("Regulation")].tolist()
print(reg_cols_ls)

['ExTRI_PMID', 'HTRI_PMID', 'TRRUST_PMID', 'TFactS_PMID', 'GOA_PMID', 'IntAct_PMID', 'SIGNOR_PMID', 'CytReg_PMID', 'GEREDB_PMID', 'NTNU_Curated_PMID', 'Pavlidis2021_PMID', 'DoRothEA_A_PMID']
['TRRUST_Regulation', 'TFactS_Regulation', 'GOA_Regulation', 'SIGNOR_Regulation', 'CytReg_Regulation', 'GEREDB_Regulation', 'NTNU_Curated_Regulation', 'Pavlidis2021_Regulation', 'DoRothEA_A_Regulation']


In [104]:
# Melt dataframes independently, and then merge them


# Melt pmid df
df_pmid = pd.melt(df_merge, id_vars= "TF_TG", value_vars= pmid_cols_ls, var_name="resource", value_name="PMID")
df_pmid["resource"] = [txt.replace("_PMID", "") for txt in df_pmid["resource"]]
df_pmid["PMID"] = [str(txt).replace(";", ",") for txt in df_pmid["PMID"]]
df_pmid[df_pmid.TF_TG=="MYC:TERT"].sort_values("resource")

,TF_TG,resource,PMID
21,MYC:TERT,CytReg,nan
33,MYC:TERT,DoRothEA_A,"11274400,12695333"
0,MYC:TERT,ExTRI,"10022128,10022128,10022128,10491298,10491298,1..."
24,MYC:TERT,GEREDB,nan
12,MYC:TERT,GOA,nan
3,MYC:TERT,HTRI,11274400|12695333
15,MYC:TERT,IntAct,nan
27,MYC:TERT,NTNU_Curated,11606399|11916966|||
30,MYC:TERT,Pavlidis2021,21627565|10022128|16880523|10022128
18,MYC:TERT,SIGNOR,nan


In [105]:
# Melt regulation df
df_reg = pd.melt(df_merge, id_vars=['TF_TG'], value_vars= reg_cols_ls, var_name="resource", value_name="regulation")
df_reg["resource"] = [txt.replace("_Regulation", "") for txt in df_reg["resource"]]
#df_reg.head(30)
df_reg[df_reg.TF_TG=="MYC:TERT"].sort_values("resource")

,TF_TG,resource,regulation
12,MYC:TERT,CytReg,NaN
24,MYC:TERT,DoRothEA_A,activation
15,MYC:TERT,GEREDB,NaN
6,MYC:TERT,GOA,activation
18,MYC:TERT,NTNU_Curated,+|-|||
21,MYC:TERT,Pavlidis2021,not_applicable|activation|activation|activation
9,MYC:TERT,SIGNOR,NaN
3,MYC:TERT,TFactS,activation
0,MYC:TERT,TRRUST,Activation|Unknown


In [100]:
# Merge into final dataframe
TF_TG_df = pd.merge(df_pmid, df_reg, how="left", on=["TF_TG", "resource"])
TF_TG_df[TF_TG_df.TF_TG=="MYC:TERT"]

,TF_TG,resource,PMID,regulation
0,MYC:TERT,ExTRI,"10022128,10022128,10022128,10491298,10491298,1...",NaN
3,MYC:TERT,HTRI,11274400|12695333,NaN
6,MYC:TERT,TRRUST,"10022128,15595642,18226852,18754863,21132266,2...",Activation|Unknown
9,MYC:TERT,TFactS,14519204,activation
12,MYC:TERT,GOA,nan,activation
15,MYC:TERT,IntAct,nan,NaN
18,MYC:TERT,SIGNOR,nan,NaN
21,MYC:TERT,CytReg,nan,NaN
24,MYC:TERT,GEREDB,nan,NaN
27,MYC:TERT,NTNU_Curated,11606399|11916966|||,+|-|||


In [16]:
# Check what is inside rows to see format: Normally PMIDs are separated by "|" to differentiate IDs associated to the different regulation modes

In [17]:
TF_TG_df["regulation"][27]

'+|-|||'

In [18]:
TF_TG_df["PMID"][27]

'11606399|11916966|||'

In [19]:
TF_TG_df["regulation"][6]

'Activation|Unknown'

In [20]:
TF_TG_df["PMID"][6]

'10022128,15595642,18226852,18754863,21132266,22207128|10637317,12941894,14611815,15958520,17706770,18093727,19912441,21553143,24705139'

In [21]:
# Can do a split of IDs and regulations modes into lists to then explode columns

In [22]:
str(TF_TG_df["PMID"][27]).split('|')

['11606399', '11916966', '', '', '']

In [23]:
str(TF_TG_df["regulation"][27]).split('|')

['+', '-', '', '', '']

In [24]:
str(TF_TG_df["PMID"][6]).split('|')

['10022128,15595642,18226852,18754863,21132266,22207128',
 '10637317,12941894,14611815,15958520,17706770,18093727,19912441,21553143,24705139']

In [25]:
# Check length after split
len(str(TF_TG_df["PMID"][6]).split('|'))

2

In [26]:
# Using specific indices could access to the number of PMIDs related to a given regulation mode
str(TF_TG_df["PMID"][6]).split('|')[0]

'10022128,15595642,18226852,18754863,21132266,22207128'

In [27]:
# IMPORTANT TO CORRECT: Here lenght is 53 as it is taking everything as string: need to convert to list
len(str(TF_TG_df["PMID"][6]).split('|')[0])

53

In [28]:
# Split IDs and regulation by "|"
TF_TG_df['PMID']= [str(txt).split('|') for txt in TF_TG_df['PMID']]
TF_TG_df['regulation']= [str(txt).split('|') for txt in TF_TG_df['regulation']]
TF_TG_df

,TF_TG,resource,PMID,regulation
0,MYC:TERT,ExTRI,"[10022128,10022128,10022128,10491298,10491298,...",[nan]
1,AP1:CAT,ExTRI,"[10022519,10329043,12036993,12538496,17935786,...",[nan]
2,SPI1:BGLAP,ExTRI,[10022617],[nan]
3,MYC:TERT,HTRI,"[11274400, 12695333]",[nan]
4,AP1:CAT,HTRI,[nan],[nan]
5,SPI1:BGLAP,HTRI,[nan],[nan]
6,MYC:TERT,TRRUST,"[10022128,15595642,18226852,18754863,21132266,...","[Activation, Unknown]"
7,AP1:CAT,TRRUST,[nan],[nan]
8,SPI1:BGLAP,TRRUST,[nan],[nan]
9,MYC:TERT,TFactS,[14519204],[activation]


In [29]:
len(TF_TG_df["PMID"][6])

2

In [30]:
len(TF_TG_df["regulation"][6])

2

In [31]:
# If further splitting by comma, can calculate lenght of PMIDs per mode of regulation
len((TF_TG_df["PMID"][6][0].split(",")))

6

In [32]:
TF_TG_df["regulation"][6][0].split(",")

['Activation']

In [33]:
len(TF_TG_df["regulation"][6][1])

7

In [47]:
pd.wide_to_long(TF_TG_df,['sur'],['TF_TG','resource'],'lol')


,,,regulation,PMID,sur
TF_TG,resource,lol,,,


In [50]:
TF_TG_df.head()

,TF_TG,resource,PMID,regulation
0,MYC:TERT,ExTRI,"[10022128,10022128,10022128,10491298,10491298,...",[nan]
1,AP1:CAT,ExTRI,"[10022519,10329043,12036993,12538496,17935786,...",[nan]
2,SPI1:BGLAP,ExTRI,[10022617],[nan]
3,MYC:TERT,HTRI,"[11274400, 12695333]",[nan]
4,AP1:CAT,HTRI,[nan],[nan]


In [106]:
test=TF_TG_df.set_index(['TF_TG','resource']).apply(pd.Series).stack().reset_index()#.drop('level_3‌​',1)
val=test[0].apply(lambda x: x.count('|'))
test[test.TF_TG=="MYC:TERT"]

,TF_TG,resource,level_2,0
0,MYC:TERT,ExTRI,PMID,"10022128,10022128,10022128,10491298,10491298,1..."
3,MYC:TERT,HTRI,PMID,11274400|12695333
6,MYC:TERT,TRRUST,PMID,"10022128,15595642,18226852,18754863,21132266,2..."
7,MYC:TERT,TRRUST,regulation,Activation|Unknown
10,MYC:TERT,TFactS,PMID,14519204
11,MYC:TERT,TFactS,regulation,activation
14,MYC:TERT,GOA,PMID,nan
15,MYC:TERT,GOA,regulation,activation
18,MYC:TERT,IntAct,PMID,nan
21,MYC:TERT,SIGNOR,PMID,nan


In [87]:
# Exploding not working as it is saying elements are not of same length, which is weird
test_explode =TF_TG_df.explode(['PMID','regulation'], ignore_index=True))
test_explode

SyntaxError: unmatched ')' (3291034091.py, line 2)

In [40]:
import pandas as pd
import numpy as np
technologies = ({'A': [["Spark","PySpark","Python"], 'Course', [], ["Java","pandas"]],
                   'B': 25000,
                   'C': [['30days','40days','35days'], np.nan, [], ['40days','55days']]})
df = pd.DataFrame(technologies)
print(df)

# Use DataFrame.explode() function
df2 = df.explode('A')
print(df2)

# Explode single column 
# Using DataFrame.explode()function
df2 = df.explode(list('A'))
print(df2)

# Explode single column & ignore_index
df2 = df.explode('A',ignore_index=True)
print(df2)

# Explode multiple columns 
# Using DataFrame.explode() function
df2 = df.explode(list('AC'))
print(df2)

# Use DataFrame.explode() Function & ignore_index
df2 = df.explode(list('A','C'), ignore_index=True)
print(df2)

                          A      B                         C
0  [Spark, PySpark, Python]  25000  [30days, 40days, 35days]
1                    Course  25000                       NaN
2                        []  25000                        []
3            [Java, pandas]  25000          [40days, 55days]
         A      B                         C
0    Spark  25000  [30days, 40days, 35days]
0  PySpark  25000  [30days, 40days, 35days]
0   Python  25000  [30days, 40days, 35days]
1   Course  25000                       NaN
2      NaN  25000                        []
3     Java  25000          [40days, 55days]
3   pandas  25000          [40days, 55days]
         A      B                         C
0    Spark  25000  [30days, 40days, 35days]
0  PySpark  25000  [30days, 40days, 35days]
0   Python  25000  [30days, 40days, 35days]
1   Course  25000                       NaN
2      NaN  25000                        []
3     Java  25000          [40days, 55days]
3   pandas  25000          [40days,

In [39]:
list('AC')

['A', 'C']